<a href="https://colab.research.google.com/github/jjcremer/NLP_Masters/blob/main/NLP202/nlp202_assignment2_1542248.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastBPE sacremoses subword_nmt #hydra-core omegaconf
#!pip install bitarray
!pip uninstall sacrebleu; pip install sacrebleu==1.5.1
!pip install fairseq

     |████████████████████████████████| 880 kB 21.8 MB/s 
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483156 sha256=a9d07e0dc3d3411db6517868f578bf7fcd15a590dc404e66319e6c700e4312c4
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=25267efb702fd6e3cf854356b5339d19a3770eb00d6bf2a3e245719bb9768bf8
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built fastBPE sacremoses
     |████████████████████████████████| 54 kB 2.8 MB/s 
     |████████████████████████████████| 1.7 MB 13.4 MB/s 
     |████████████████████████████████| 147 kB 63.1 MB/s 
     |████████████████████████████████| 74 kB 3.6 MB/s 
     |████████████████████████████████| 112 kB 68.7 MB/s 
     |████████████████████████████████| 596 kB 66.5 MB/s 
  Created wheel for an

In [ ]:
import torch
from fairseq.models.transformer import TransformerModel

In [ ]:
#!bash prepare-iwslt14.sh
!ls
!unzip orig.zip
!bash prepare-iwslt13.sh
#!bash prepare-iwslt13-nobpe.sh

orig.zip  prepare-iwslt13-nobpe.sh  prepare-iwslt13.sh	sample_data
Archive:  orig.zip
   creating: orig/
  inflating: __MACOSX/._orig         
   creating: orig/fr-en/
  inflating: __MACOSX/orig/._fr-en   
  inflating: orig/.DS_Store          
  inflating: __MACOSX/orig/._.DS_Store  
  inflating: orig/fr-en.tgz          
  inflating: __MACOSX/orig/._fr-en.tgz  
  inflating: orig/fr-en/IWSLT13.TED.dev2010.fr-en.en.xml  
  inflating: __MACOSX/orig/fr-en/._IWSLT13.TED.dev2010.fr-en.en.xml  
  inflating: orig/fr-en/IWSLT13.TED.dev2010.fr-en.fr.xml  
  inflating: __MACOSX/orig/fr-en/._IWSLT13.TED.dev2010.fr-en.fr.xml  
  inflating: orig/fr-en/train.fr     
  inflating: __MACOSX/orig/fr-en/._train.fr  
  inflating: orig/fr-en/README       
  inflating: __MACOSX/orig/fr-en/._README  
  inflating: orig/fr-en/train.en     
  inflating: __MACOSX/orig/fr-en/._train.en  
  inflating: orig/fr-en/IWSLT13.TED.tst2010.fr-en.en.xml  
  inflating: __MACOSX/orig/fr-en/._IWSLT13.TED.tst2010.fr-en.en.xml  

In [ ]:
!ls
!TEXT=iwslt13.tokenized.fr-en
'''
!fairseq-preprocess --source-lang fr --target-lang en \
    --trainpref iwslt13.tokenized.fr-en/tmp/train --validpref iwslt13.tokenized.fr-en/tmp/valid --testpref iwslt13.tokenized.fr-en/tmp/test \
    --destdir data-bin/iwslt13.tokenized.fr-en \
    --workers 20 \
'''
!fairseq-preprocess --source-lang fr --target-lang en \
    --trainpref iwslt13.tokenized.fr-en/train --validpref iwslt13.tokenized.fr-en/valid --testpref iwslt13.tokenized.fr-en/test \
    --destdir data-bin/iwslt13.tokenized.fr-en \
    --workers 20 \

iwslt13.tokenized.fr-en  orig			   prepare-iwslt13.sh
__MACOSX		 orig.zip		   sample_data
mosesdecoder		 prepare-iwslt13-nobpe.sh  subword-nmt
2022-05-11 17:57:56 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/iwslt13.tokenized.fr-en', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang='fr', srcdict=None, target_lang='en', task='translati

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train \
    data-bin/iwslt13.tokenized.fr-en \
    --max-epoch 10 \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 4000 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir checkpoints/transformer_bpe

2022-05-11 20:12:11 | INFO | fairseq_cli.train | Namespace(activation_dropout=0.0, activation_fn='relu', adam_betas='(0.9, 0.98)', adam_eps=1e-08, adaptive_input=False, adaptive_softmax_cutoff=None, adaptive_softmax_dropout=0, all_gather_list_size=16384, arch='transformer', attention_dropout=0.0, batch_size=None, batch_size_valid=None, best_checkpoint_metric='bleu', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.0, cpu=False, criterion='label_smoothed_cross_entropy', cross_self_attention=False, curriculum=0, data='data-bin/iwslt13.tokenized.fr-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention_heads=8, decoder_embed_dim=512, decoder_embed_path=None, decoder_ffn_embed_dim=2048, decoder_input_dim=512, decoder_layerdrop=0, decoder_layers=6, decoder_layers_to_keep=None, decoder_learned_pos=False, decoder_normalize_before=False, decoder_output_dim=512, device_id=0, disable_validatio

In [ ]:
'''
!fairseq-generate data-bin/iwslt13.tokenized.fr-en \
    --path checkpoints/transformer_nobpe/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe \
    --scoring sacrebleu \
    --results-path results/transformer_nobpe
'''
!fairseq-generate data-bin/iwslt13.tokenized.fr-en \
    --path checkpoints/transformer_bpe/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe \
    --scoring sacrebleu 
    #--results-path results/transformer_bpe_3e_4

Streaming output truncated to the last 5000 lines.
D-1925	-1.9858037233352661	they &apos;re going to be in a city of a city , where the ground is where the more likely to build their own ones .
P-1925	-0.0574 -1.0487 -4.6289 -0.4044 -2.6142 -0.6120 -0.6048 -1.2212 -2.0495 -4.5262 -2.9661 -2.4138 -0.0265 -0.5638 -4.3910 -1.1118 -3.1745 -0.4201 -4.1699 -3.6899 -0.1386 -0.3170 -0.7298 -4.8420 -6.5035 -0.3292 -0.0618
S-1555	et si on ne connait pas exactement le degré de gravité de ces effets , on sait qu&apos; ils seront extrêmement graves .
T-1555	and there &apos;s certainly uncertainty about how bad those effects will be , but they will be extremely bad .
H-1555	-1.404492974281311	and if we don &apos;t know exactly exactly exactly exactly the amount of these rules , you know , they &apos;re going to be very difficult .
D-1555	-1.404492974281311	and if we don &apos;t know exactly exactly exactly exactly the amount of these rules , you know , they &apos;re going to be very difficult .
P-15

In [ ]:
!mkdir -p checkpoints/fconv_iwslt_fr_en
!fairseq-train \
    data-bin/iwslt13.tokenized.fr-en/ \
    --max-epoch 10 \
    --arch fconv \
    --dropout 0.2 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --optimizer nag --clip-norm 0.1 \
    --lr 0.5 --lr-scheduler fixed --force-anneal 50 \
    --max-tokens 4000 \
    --save-dir checkpoints/fconv_iwslt_fr_en

2022-05-10 01:16:12 | INFO | fairseq_cli.train | Namespace(all_gather_list_size=16384, arch='fconv', batch_size=None, batch_size_valid=None, best_checkpoint_metric='loss', bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', clip_norm=0.1, cpu=False, criterion='label_smoothed_cross_entropy', curriculum=0, data='data-bin/iwslt13.tokenized.fr-en/', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoder_attention='True', decoder_embed_dim=512, decoder_embed_path=None, decoder_layers='[(512, 3)] * 20', decoder_out_embed_dim=256, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_num_procs=1, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', dropout=0.2, empty_cache_freq=0, encoder_embed_dim=512, encoder_embed_path=None, encoder_layers='[(512, 3)] * 20', eval_bleu=False, eval_bleu_args=None,

In [ ]:
!fairseq-generate data-bin/iwslt13.tokenized.fr-en \
    --path checkpoints/fconv_iwslt_fr_en/checkpoint_best.pt \
    --batch-size 128 --beam 5 --remove-bpe \
    --scoring sacrebleu

2022-05-10 01:19:50 | INFO | fairseq_cli.generate | Namespace(all_gather_list_size=16384, batch_size=128, batch_size_valid=128, beam=5, bf16=False, bpe=None, broadcast_buffers=False, bucket_cap_mb=25, checkpoint_shard_count=1, checkpoint_suffix='', constraints=None, cpu=False, criterion='cross_entropy', curriculum=0, data='data-bin/iwslt13.tokenized.fr-en', data_buffer_size=10, dataset_impl=None, ddp_backend='c10d', decoding_format=None, device_id=0, disable_validation=False, distributed_backend='nccl', distributed_init_method=None, distributed_no_spawn=False, distributed_port=-1, distributed_rank=0, distributed_world_size=1, distributed_wrapper='DDP', diverse_beam_groups=-1, diverse_beam_strength=0.5, diversity_rate=-1.0, empty_cache_freq=0, eval_bleu=False, eval_bleu_args=None, eval_bleu_detok='space', eval_bleu_detok_args=None, eval_bleu_print_samples=False, eval_bleu_remove_bpe=None, eval_tokenized_bleu=False, fast_stat_sync=False, find_unused_parameters=False, fix_batches_to_gpus=